<a href="https://colab.research.google.com/github/mdapoy/Machine-Learning-week-8-16/blob/main/Ch16_Natural_Language_Processing_with_RNNs_and_Attention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Laporan Analisis Bab

**Judul Buku:** Hands-on Machine Learning with Scikit-Learn, Keras, and TensorFlow
**Penulis:** Aurélien Géron
**Edisi:** Kedua, Diperbarui untuk TensorFlow 2

---

## Bab 16: Natural Language Processing with RNNs and Attention

**I. Pendahuluan**
Bab 16 membahas aplikasi *Natural Language Processing* (NLP) menggunakan *Recurrent Neural Networks* (RNNs) dan *Attention Mechanisms*. Penulis memulai dengan merujuk pada *Turing Test* Alan Turing, yang menekankan pentingnya penguasaan bahasa sebagai kemampuan kognitif manusia yang luar biasa. Bab ini melanjutkan eksplorasi RNN dari Bab 15, dimulai dengan RNN karakter untuk pembuatan teks, kemudian beralih ke model tingkat kata untuk analisis sentimen, dan akhirnya membahas *Neural Machine Translation* (NMT) menggunakan arsitektur *Encoder-Decoder*. Bagian kedua bab ini secara khusus menyoroti *Attention Mechanisms* dan arsitektur *Transformer* yang sangat sukses, serta inovasi terbaru dalam model bahasa seperti GPT-2 dan BERT.

**II. Generating Shakespearean Text Using a Character RNN**
Penulis memperkenalkan *Character RNN* (Char-RNN) sebagai model yang dapat memprediksi karakter berikutnya dalam sebuah kalimat dan digunakan untuk menghasilkan teks baru.

**Konsep dan Implementasi:**
* **Dataset:** Menggunakan semua karya Shakespeare. Setiap karakter dikodekan sebagai bilangan bulat menggunakan kelas `keras.preprocessing.text.Tokenizer` dengan `char_level=True`.
* **Pembagian Dataset Sekuensial:** Penting untuk menghindari tumpang tindih antara set pelatihan, validasi, dan pengujian. Umumnya, pembagian dilakukan berdasarkan waktu, meskipun dapat juga berdasarkan dimensi lain jika data tidak berkorelasi kuat (misalnya, antar perusahaan, tetapi ada risiko bias jika ada korelasi yang tersembunyi). Asumsi bahwa deret waktu stasioner (*stationary*) seringkali menjadi dasar pembagian berdasarkan waktu.
* **Pemotongan Dataset Sekuensial menjadi Banyak Jendela:** Sekuens karakter yang panjang dikonversi menjadi banyak jendela teks yang lebih kecil menggunakan metode `window()` dataset. Ini dikenal sebagai *truncated backpropagation through time*. `shift=1` digunakan untuk jendela yang tumpang tindih guna mendapatkan set pelatihan terbesar, dan `drop_remainder=True` untuk memastikan semua jendela memiliki panjang yang sama.
* **Flattening Dataset:** Metode `flat_map()` digunakan untuk mengubah dataset bersarang menjadi dataset datar yang berisi tensor (bukan dataset).
* **Pengacakan dan Batching:** Jendela-jendela ini kemudian diacak (`shuffle()`) untuk memastikan *instance* pelatihan independen dan terdistribusi secara identik (IID), lalu di-*batch* (`batch()`). Input (100 karakter pertama) dipisahkan dari target (karakter terakhir).
* **Pengkodean Fitur Input:** Setiap karakter dikodekan menggunakan vektor *one-hot* karena jumlah karakter unik yang relatif sedikit (39).
* **Prefetching:** `prefetch(1)` ditambahkan untuk meningkatkan efisiensi pipeline input.
* **Model Char-RNN:** Dibangun menggunakan RNN dengan dua lapisan GRU (masing-masing 128 unit) dan *dropout* 20% pada input dan *hidden state*. Lapisan output adalah lapisan `TimeDistributed(Dense)` dengan 39 unit dan fungsi aktivasi *softmax*. Model ini dikompilasi dengan *loss* `"sparse_categorical_crossentropy"` dan *optimizer* "adam".
* **Menggunakan Model Char-RNN:** Fungsi `preprocess()` dibuat untuk mengodekan teks input ke format yang diharapkan model.
* **Menghasilkan Teks Shakespearean Palsu:** Untuk menghasilkan teks baru, model memprediksi probabilitas untuk setiap karakter berikutnya. Karakter berikutnya dipilih secara acak berdasarkan probabilitas ini menggunakan `tf.random.categorical()`. Parameter `temperature` dapat disesuaikan untuk mengontrol keragaman teks yang dihasilkan.

**III. Stateful RNN**
Berbeda dengan RNN *stateless* yang membuang *hidden state* setelah setiap *batch*, RNN *stateful* mempertahankan *final state* setelah memproses satu *batch* pelatihan dan menggunakannya sebagai *initial state* untuk *batch* pelatihan berikutnya. Ini memungkinkan model untuk mempelajari pola jangka panjang meskipun hanya melakukan *backpropagation* melalui sekuens pendek.

**Implementasi Stateful RNN:**
* Input sekuens harus berurutan dan tidak tumpang tindih. `shift=n_steps` digunakan pada `window()` dan `shuffle()` tidak dipanggil.
* `stateful=True` diatur pada setiap lapisan rekuren.
* `batch_input_shape` diatur pada lapisan pertama karena RNN *stateful* perlu mengetahui ukuran *batch*.
* *Callback* `ResetStatesCallback` digunakan untuk mengatur ulang *state* di akhir setiap *epoch*.
* Setelah pelatihan, model *stateful* hanya dapat digunakan untuk prediksi *batch* dengan ukuran yang sama dengan yang digunakan selama pelatihan. Untuk mengatasi batasan ini, model *stateless* identik dapat dibuat dan bobot model *stateful* disalin ke dalamnya.

**IV. Sentiment Analysis**
Bab ini beralih ke model tingkat kata untuk tugas analisis sentimen menggunakan dataset ulasan film IMDb.

**Konsep dan Implementasi:**
* **Dataset IMDb:** Terdiri dari 50.000 ulasan film yang sudah di-*preprocess* (dikonversi menjadi *array* bilangan bulat yang mewakili kata-kata), dengan label biner (negatif/positif). ID 0, 1, dan 2 adalah token khusus untuk *padding*, *start-of-sequence* (SOS), dan kata-kata yang tidak diketahui.
* **Preprosessing Teks:** Fungsi *preprocessing* kustom dibuat menggunakan operasi `tf.strings` untuk memotong ulasan, mengganti *tag* HTML, menghapus karakter non-alfabet, dan memecah ulasan menjadi kata-kata (mengonversi ke *ragged tensor* lalu *dense tensor* dengan *padding*).
* **Pembuatan Vocabulary:** *Vocabulary* dibangun dengan menghitung kemunculan kata-kata menggunakan `collections.Counter` pada *training set* yang sudah di-*preprocess*. *Vocabulary* dipotong untuk hanya menyimpan 10.000 kata yang paling umum.
* **Pemetaan Kata ke ID:** *Lookup table* (`tf.lookup.StaticVocabularyTable`) dibuat untuk memetakan setiap kata ke ID-nya dalam *vocabulary*, termasuk *out-of-vocabulary* (OOV) *bucket* untuk kata-kata yang tidak dikenal.
* **Model Analisis Sentimen:**
    * **Lapisan Embedding:** Lapisan pertama adalah `keras.layers.Embedding` yang mengubah ID kata menjadi *embedding* (vektor padat yang dapat dilatih). Outputnya adalah tensor 3D `[batch size, time steps, embedding size]`.
    * **Lapisan GRU:** Dua lapisan GRU diikuti setelah lapisan *embedding*. Lapisan GRU kedua hanya mengembalikan output dari langkah waktu terakhir.
    * **Lapisan Output:** Neuron tunggal dengan fungsi aktivasi *sigmoid* untuk memprediksi probabilitas sentimen positif.
* **Masking:** `mask_zero=True` diatur pada lapisan *Embedding* untuk secara otomatis mengabaikan token *padding* (ID 0) oleh lapisan-lapisan berikutnya, terutama lapisan rekuren. Ini memungkinkan model fokus pada data yang relevan. Masking juga dapat ditangani secara manual menggunakan *Functional API* atau *Subclassing API*.
* **Menggunakan Pretrained Embeddings:** Proyek TensorFlow Hub memungkinkan penggunaan komponen model *pretrained*, seperti *nnlm-en-dim50 sentence embedding module*, yang mengodekan string input sebagai vektor 50-dimensi. Ini memungkinkan model analisis sentimen untuk menggunakan *embedding* kata yang sudah dilatih pada korpus teks yang sangat besar, meningkatkan kinerja bahkan dengan data pelatihan yang terbatas.

**V. An Encoder–Decoder Network for Neural Machine Translation**
Model terjemahan mesin saraf (NMT) dibahas, yang menerjemahkan kalimat dari satu bahasa ke bahasa lain.

**Arsitektur dan Proses:**
* **Encoder:** Menerima kalimat input (misalnya, bahasa Inggris), seringkali dibalik, dan mengodekannya menjadi representasi vektor.
* **Decoder:** Menerima output dari *encoder* dan menghasilkan terjemahan (misalnya, bahasa Prancis). Input ke *decoder* adalah kata yang seharusnya dikeluarkan pada langkah waktu sebelumnya (token SOS untuk kata pertama).
* **Word Embeddings:** Setiap kata direpresentasikan oleh ID-nya dan kemudian diubah menjadi *word embedding* yang merupakan input sebenarnya ke *encoder* dan *decoder*.
* **Prediksi:** Pada setiap langkah waktu, *decoder* menghasilkan skor untuk setiap kata dalam *vocabulary* output, yang kemudian diubah menjadi probabilitas oleh lapisan *softmax*. Kata dengan probabilitas tertinggi dipilih.
* **Inference Time:** Pada waktu inferensi, *decoder* diberi kata yang dikeluarkan pada langkah sebelumnya sebagai input.
* **Penanganan Panjang Kalimat Variabel:** *Padding* digunakan untuk membuat semua kalimat dalam satu *batch* memiliki panjang yang sama. Kalimat-kalimat dikelompokkan ke dalam *bucket* dengan panjang yang serupa (`tf.data.experimental.bucket_by_sequence_length()`).
* **Masking Output:** Output setelah token EOS harus diabaikan dan tidak berkontribusi pada *loss*.
* **Menangani Vocabulary Output Besar:** Untuk *vocabulary* output yang besar, *sampled softmax* (`tf.nn.sampled_softmax_loss()`) digunakan selama pelatihan untuk menghemat komputasi.
* **Implementasi dengan TensorFlow Addons:** Proyek ini menyediakan alat *sequence-to-sequence* seperti `tfa.seq2seq.sampler.TrainingSampler` dan `tfa.seq2seq.basic_decoder.BasicDecoder` untuk membangun *Encoder-Decoder*.

**VI. Bidirectional RNNs**
Untuk banyak tugas NLP, seperti NMT, lebih baik melihat kata-kata berikutnya sebelum mengodekan kata tertentu. Lapisan rekuren *bidirectional* melakukannya dengan menjalankan dua lapisan rekuren pada input yang sama: satu dari kiri ke kanan dan satu dari kanan ke kiri. Outputnya kemudian digabungkan (biasanya dengan mengkonkatenaasinya). `keras.layers.Bidirectional` digunakan untuk mengimplementasikannya.

**VII. Beam Search**
Teknik ini digunakan untuk meningkatkan kinerja model *Encoder-Decoder* yang sudah dilatih, terutama untuk menghasilkan terjemahan yang lebih baik. Daripada secara rakus memilih kata berikutnya yang paling mungkin di setiap langkah, *beam search* melacak daftar pendek `k` kalimat yang paling menjanjikan (lebar *beam* `k`) dan mencoba memperpanjangnya dengan satu kata, hanya menyimpan `k` kalimat yang paling mungkin. Implementasi dapat dilakukan dengan `tfa.seq2seq.beam_search_decoder.BeamSearchDecoder`.

**VIII. Attention Mechanisms**
*Attention Mechanisms* adalah komponen jaringan saraf yang belajar memilih bagian input yang harus difokuskan oleh model di setiap langkah waktu. Ini secara signifikan memperpendek jalur dari kata input ke terjemahannya, mengatasi keterbatasan memori jangka pendek RNN.

**Jenis Attention:**
* **Bahdanau Attention (Concatenative/Additive Attention):** *Alignment model* (lapisan *attention* kecil) menghitung skor (atau energi) untuk setiap output *encoder* berdasarkan gabungan output *encoder* dan *hidden state* *decoder* sebelumnya. Skor-skor ini kemudian dinormalisasi oleh *softmax* untuk mendapatkan bobot akhir.
* **Luong Attention (Multiplicative Attention):** Menghitung produk titik (*dot product*) antara output *encoder* dan *hidden state* *decoder* sebelumnya sebagai ukuran kesamaan. Hasil skor dinormalisasi oleh *softmax* untuk mendapatkan bobot akhir. Varian "general" Luong attention juga ada.

**Aplikasi Lain Attention:**
* **Visual Attention:** Digunakan dalam pembuatan *caption* gambar, di mana *decoder* RNN dengan *attention mechanism* belajar fokus pada bagian gambar yang relevan saat menghasilkan setiap kata.
* **Explainability:** Memungkinkan pemahaman yang lebih baik tentang mengapa model menghasilkan output tertentu, sangat membantu untuk *debugging* dan interpretasi model.

**IX. Attention Is All You Need: The Transformer Architecture**
Diperkenalkan pada tahun 2017, *Transformer* merevolusi NMT dengan hanya menggunakan *attention mechanisms* (ditambah *embedding layers*, *dense layers*, *normalization layers*, dll.) tanpa lapisan rekuren atau konvolusional.

**Arsitektur Transformer:**
* **Encoder:** Menerima *batch* kalimat input dan mengodekan setiap kata menjadi representasi dimensi tinggi.
* **Decoder:** Menerima kalimat target (digeser satu langkah waktu ke kanan) dan output dari *encoder*, lalu menghasilkan probabilitas untuk setiap kata berikutnya.
* **Positional Embeddings:** Vektor padat yang mengodekan posisi kata dalam kalimat, ditambahkan ke *word embedding*. Ini memberikan informasi posisi yang penting karena lapisan *Multi-Head Attention* tidak mempertimbangkan urutan kata. Dapat berupa *fixed positional embeddings* (sine/cosine functions) atau *learned positional embeddings*.
* **Multi-Head Attention:** Inti dari *Transformer*. Terdiri dari beberapa lapisan *Scaled Dot-Product Attention*, masing-masing didahului oleh transformasi linier dari nilai, kunci, dan kueri. Ini memungkinkan model untuk fokus pada berbagai karakteristik kata (misalnya, menjadi kata kerja, atau berada dalam *past tense*) dalam subruang yang berbeda.

**X. Recent Innovations in Language Models (2018-2019)**
Tahun 2018 disebut sebagai "momen ImageNet untuk NLP" karena kemajuan yang luar biasa.
* **ELMo (Embeddings from Language Models) (2018):** Memperkenalkan *contextualized word embeddings* yang dipelajari dari *internal state* model bahasa *bidirectional* yang dalam. *Embedding* kata bervariasi berdasarkan konteksnya.
* **ULMFiT (Universal Language Model Fine-Tuning) (2018):** Menunjukkan efektivitas *unsupervised pretraining* untuk NLP. Model bahasa LSTM dilatih menggunakan *self-supervised learning* pada korpus teks besar, lalu di-*fine-tune* untuk berbagai tugas, secara signifikan mengurangi *error rate*. Dapat mencapai kinerja yang sama dengan model yang dilatih dari awal hanya dengan 100 contoh berlabel.
* **GPT (Generative Pre-Training) (2018) & GPT-2 (2019):** Diperkenalkan oleh OpenAI. Menggunakan arsitektur *Transformer*-like (hanya lapisan *Masked Multi-Head Attention*) yang dilatih secara *unsupervised pretraining* pada dataset besar. GPT-2, dengan 1.5 miliar parameter, menunjukkan kemampuan *zero-shot learning* (kinerja baik tanpa *fine-tuning* pada tugas spesifik).
* **BERT (Bidirectional Encoder Representations from Transformers) (2018):** Diperkenalkan oleh Google. Mirip dengan GPT tetapi menggunakan lapisan *Multi-Head Attention* non-masking, menjadikannya *bidirectional*. Kekuatannya berasal dari dua tugas *pretraining*:
    * **Masked Language Model (MLM):** Memprediksi kata-kata yang di-*mask* dalam kalimat.
    * **Next Sentence Prediction (NSP):** Memprediksi apakah dua kalimat berurutan atau tidak.

**Tren Terkini:** Inovasi utama meliputi *subword tokenization* yang lebih baik, pergeseran dari LSTM ke *Transformer*, dan *pretraining* model bahasa universal menggunakan *self-supervised learning*. Bidang ini terus bergerak cepat, dengan potensi kembalinya CNNs atau RNNs dengan inovasi baru.

**X. Kesimpulan**
Bab ini memberikan wawasan mendalam tentang aplikasi NLP, mulai dari pembuatan teks sederhana hingga terjemahan mesin yang kompleks. Evolusi dari RNN dasar ke arsitektur canggih seperti LSTM, GRU, dan terutama *Transformer* ditunjukkan secara detail, menyoroti bagaimana setiap inovasi mengatasi keterbatasan sebelumnya. Pembahasan tentang *attention mechanisms* dan model bahasa *pretrained* seperti BERT dan GPT-2 menunjukkan terobosan signifikan dalam NLP beberapa tahun terakhir, yang telah mengubah cara kita mendekati tugas-tugas bahasa alami.

# REPRODUCE CODE